# SageMaker를 사용한 ML을 위한 데이터 전처리
머신 러닝 프로젝트를 진행할 때, 모델 학습 요구사항에 맞는 데이터셋을 구축하기 위해 전처리 및 후처리 작업을 수행하는 것이 일반적입니다. 데이터 과학자들은 종종 머신 러닝 모델 학습을 위한 데이터를 준비하기 위해 노트북에서 전처리 스크립트를 개발합니다. 이 워크숍에서는 플레이어 이탈 가능성을 예측하는 분류기를 학습시키기 위해 합성 비디오 게임 데이터셋을 사용하고 있습니다. 이 노트북의 데이터 전처리는 [XGBoost 분류기](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html)를 학습시키기 위한 데이터 구조를 준비하는 데 중점을 둡니다.


![preprocessing notebook](img/sagemaker-mlops-notebook-preprocessing.jpg)

In [ ]:
%pip install scikit-learn "pandas>=2.0.0" s3fs==0.4.2 sagemaker xgboost mlflow==2.13.2 sagemaker-mlflow==0.1.0

노트북에서 사용할 필수 패키지들을 가져옵니다

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import boto3
from sagemaker.session import Session
from sagemaker.feature_store.feature_store import FeatureStore
from sagemaker.feature_store.feature_group import FeatureGroup
import numpy as np
import sagemaker
import mlflow
import pandas as pd
from time import gmtime, strftime
import os
import json
from botocore.exceptions import ClientError

## 헬퍼 함수

In [4]:
def download_from_s3(s3_client, local_file_path, bucket_name, s3_file_path):
    try:
        # Download the file
        s3_client.download_file(bucket_name, s3_file_path, local_file_path)
        print(f"File downloaded successfully to {local_file_path}")
        return True
    except ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            print(f"An error occurred: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

def upload_to_s3(s3_client, local_file_path, bucket_name, s3_file_path=None):
    # If S3 file path is not specified, use the basename of the local file
    if s3_file_path is None:
        s3_file_path = os.path.basename(local_file_path)

    try:
        # Upload the file
        s3_client.upload_file(local_file_path, bucket_name, s3_file_path)
        print(f"File {local_file_path} uploaded successfully to {bucket_name}/{s3_file_path}")
        return True
    except ClientError as e:
        print(f"ClientError: {e}")
        return False
    except FileNotFoundError:
        print(f"The file {local_file_path} was not found")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False
        
def write_params(s3_client, step_name, params, notebook_param_s3_bucket_prefix):
    local_file_path = f"{step_name}.json"
    with open(local_file_path, "w") as f:
        f.write(json.dumps(params))
    base_local_file_path = os.path.basename(local_file_path)
    bucket_name = notebook_param_s3_bucket_prefix.split("/")[2] # Format: s3://<bucket_name>/..
    s3_file_path = os.path.join("/".join(notebook_param_s3_bucket_prefix.split("/")[3:]), base_local_file_path)
    upload_to_s3(s3_client, local_file_path, bucket_name, s3_file_path)
    
def read_params(s3_client, notebook_param_s3_bucket_prefix, step_name):
    local_file_path = f"{step_name}.json"
    base_local_file_path = os.path.basename(local_file_path)
    bucket_name = notebook_param_s3_bucket_prefix.split("/")[2] # Format: s3://<bucket_name>/..
    s3_file_path = os.path.join("/".join(notebook_param_s3_bucket_prefix.split("/")[3:]),  base_local_file_path)
    downloaded = download_from_s3(s3_client, local_file_path, bucket_name, s3_file_path)
    with open(local_file_path, "r") as f:
        data = f.read()
        params = json.loads(data)
    return params


전처리 작업을 위한 상수를 정의합니다.

In [5]:
target_col = "player_churn"

# 변수/파라미터 초기화
다음 변수들은 노트북 전체에서 사용되는 특정 변수로 이 셀에서 정의됩니다. 하드코딩된 값 외에도, 이러한 변수들은 SageMaker Pipeline 작업과 같이 노트북이 원격으로 실행되도록 예약될 때 파라미터로 노트북에 전달될 수 있습니다. 다음 실습에서 이 노트북에 파라미터를 전달하는 방법에 대해 자세히 알아보겠습니다. 다음 셀에서 정의된 변수들은 SageMaker Project를 통한 CICD 파이프라인을 통해 예약될 때 다른 값으로 업데이트될 수 있습니다. 노트북 매개변수화에 대한 자세한 정보는 [이 문서](https://docs.aws.amazon.com/sagemaker/latest/dg/notebook-auto-run-troubleshoot-override.html)를 참조하세요.

특히, SageMaker Studio에서 이 노트북을 성공적으로 실행하기 위해 얻어야 할 2개의 변수가 있습니다.

* feature_group_name
* mlflow_tracking_server_arn


다음 섹션에서는 SageMaker Studio Launcher를 통해 이러한 변수들을 얻는 방법을 설명합니다.

### 피처 스토어 그룹
피처 스토어 수집 실습(lab1)을 완료했다면, SageMaker 피처 그룹이 생성되어 있을 것입니다. 피처 그룹 이름을 얻으려면, SageMaker Studio 런처로 이동한 다음 왼쪽 창에서 `Data` 드롭다운을 선택하고, `Feature Store`를 선택한 후 다음 다이어그램과 같이 오른쪽 창에서 적절한 피처 스토어 그룹 이름을 찾으세요:

![studio feature group console](img/sagemaker-studio-feature-group-console.jpg)

### MLFlow 트래킹 서버
머신 러닝은 다양한 데이터, 알고리즘 및 파라미터 조합을 실험하고 모델 정확도에 미치는 영향을 관찰하는 반복적인 프로세스입니다. ML 실험의 반복적인 특성으로 인해 수많은 모델 학습 실행과 버전이 생성되어 최고 성능의 모델과 그 구성을 추적하기 어렵게 만듭니다. 생성형 인공지능(생성형 AI)을 사용하면 실험의 복잡성이 더욱 증가하는데, 이는 모델 미세 조정뿐만 아니라 창의적이고 다양한 출력을 탐색하는 것도 포함하기 때문입니다. 연구자들은 생성된 콘텐츠의 품질과 창의성을 최적화하기 위해 하이퍼파라미터를 조정하고, 적합한 모델 아키텍처를 선택하며, 다양한 데이터셋을 큐레이션해야 합니다. 생성형 AI 모델을 평가하려면 정량적 및 정성적 메트릭이 모두 필요하며, 이는 실험 프로세스에 또 다른 복잡성 계층을 추가합니다.

이 워크숍 전반에 걸쳐, MLflow를 Amazon SageMaker와 통합하여 반복적인 ML 실험을 추적, 구성, 보기, 분석 및 비교하여 비교 인사이트를 얻고 최고 성능의 모델을 등록하고 배포할 것입니다.

SageMaker MLFlow 통합에 대해 자세히 알아보려면 이 문서를 참조하세요.


이 노트북 전에 `00-start-here.ipynb`를 완료했다면, 사용할 수 있는 MLFlow 트래킹 서버가 프로비저닝되어 있을 것입니다. 트래킹 서버의 ARN을 얻으려면, SageMaker Studio Launcher로 이동하여 런처 왼쪽 상단의 애플리케이션 목록에서 `MLFlow`를 선택하고, 적절한 트래킹 서버를 선택한 다음, 다음 다이어그램과 같이 `Configuration` 섹션에서 ARN을 찾으세요:

![mlflow tracking](img/sagemaker-studio-launcher-mlflow-console.jpg)

In [6]:
region = "us-east-1"
os.environ["AWS_DEFAULT_REGION"] = region
boto_session = boto3.Session(region_name=region)

# Sagemaker session
sess = sagemaker.Session(boto_session=boto_session)
bucket_name = sess.default_bucket()
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)
bucket_prefix = "player-churn/xgboost"
notebook_param_s3_bucket_prefix=f"s3://{bucket_name}/{bucket_prefix}/params"
experiment_name = "player-churn-model-experiment"
feature_group_name = "" # Replace the feature group name created in the previous lab
mlflow_tracking_server_arn = "" # Provide a valid mlflow tracking server ARN. You can find the value in the output from 00-start-here.ipynb
run_id = None

In [7]:
assert len(feature_group_name) > 0
assert len(mlflow_tracking_server_arn) > 0

다음 셀은 MLFlow 트래킹 서버를 이 전처리 작업과 통합합니다.

In [8]:
suffix = strftime('%d-%H-%M-%S', gmtime())
mlflow.set_tracking_uri(mlflow_tracking_server_arn)
experiment = mlflow.set_experiment(experiment_name=experiment_name)
run = mlflow.start_run(run_id=run_id) if run_id else mlflow.start_run(run_name=f"processing-{suffix}", nested=True)

In [15]:
# Create FeatureStore session object
feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

feature_store = FeatureStore(sagemaker_session=feature_store_session)
dataset_feature_group = FeatureGroup(feature_group_name)

query_output_s3_path = f's3://{bucket_name}/data/query_results/'
# Create dataset builder to retrieve the most recent version of each record
builder = feature_store.create_dataset(
    base=dataset_feature_group,
    # included_feature_names=inlcuded_feature_names,
    output_path=query_output_s3_path,
).with_number_of_recent_records_by_record_identifier(1)

player_churn_fs_df, query = builder.to_dataframe()
player_churn_fs_df.head()

<div class="alert alert-info"><b>중요</b>: 💡 피처 스토어 그룹이 오프라인 스토어로 생성되었기 때문에, SageMaker SDK를 사용하여 데이터를 가져오는 데 지연(최대 15분)이 있을 수 있습니다. 위 셀이 아무것도 반환하지 않았다면, 몇 분 기다린 후 쿼리에서 샘플 데이터가 반환되는지 확인하기 위해 위 셀을 다시 실행해야 합니다.
</div>

# 데이터 처리 및 학습 데이터 분할
이전 단계에서 모델 학습을 위해 SageMaker Feature Store에서 필요한 데이터셋을 추출했습니다. 우리의 사용 사례에서는 플레이어 이탈을 예측하는 분류기를 학습시키기 위해 강력하고 효율적인 ML 알고리즘인 [XGBoost](https://xgboost.readthedocs.io/en/stable/)를 활용할 것입니다. XGBoost의 데이터 구조는 [여기](https://xgboost.readthedocs.io/en/stable/tutorials/input_format.html#id1)에서 찾을 수 있습니다. 다음 섹션에서는 XGBoost 모델의 요구 사항을 충족하도록 데이터셋 형식을 지정하겠습니다.

In [46]:
# Removing the columns from Feature Store not relevant to training a model.
df_model_data = player_churn_fs_df.drop( columns = ["player_id", "event_time"])

# Reorganize the data structure to make the label column as the first column in the dataset.
df_model_data = pd.concat([df_model_data["player_churn"], df_model_data.drop(columns=["player_churn"])], axis=1)

In [ ]:
# Shuffle and splitting dataset
train_data, validation_data, test_data = np.split(
    df_model_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(df_model_data)), int(0.9 * len(df_model_data))],
)

print(f"Data split > train:{train_data.shape} | validation:{validation_data.shape} | test:{test_data.shape}")

In [ ]:
train_data_output_s3_path = f"s3://{bucket_name}/{bucket_prefix}/train/train.csv"
validation_data_output_s3_path = f"s3://{bucket_name}/{bucket_prefix}/validation/validation.csv"
test_x_data_output_s3_path = f"s3://{bucket_name}/{bucket_prefix}/test/test_x.csv"
test_y_data_output_s3_path = f"s3://{bucket_name}/{bucket_prefix}/test/test_y.csv"
baseline_data_output_s3_path = f"s3://{bucket_name}/{bucket_prefix}/baseline/baseline.csv"

train_data.to_csv(train_data_output_s3_path, index=False, header=False)
validation_data.to_csv(validation_data_output_s3_path, index=False, header=False)
test_data[target_col].to_csv(test_y_data_output_s3_path, index=False, header=False)
test_data.drop([target_col], axis=1).to_csv(test_x_data_output_s3_path, index=False, header=False)

# We need the baseline dataset for model monitoring
df_model_data.drop([target_col], axis=1).to_csv(baseline_data_output_s3_path, index=False, header=False)

# 파라미터 저장
다음 셀에서는 관련 파라미터를 S3 버킷에 저장하여 이후 단계에서 다른 단계로 전달할 수 있도록 하겠습니다.

In [49]:
params = {}
params['train_data'] = train_data_output_s3_path
params['validation_data'] = validation_data_output_s3_path
params['test_x_data'] = test_x_data_output_s3_path
params['test_y_data'] = test_y_data_output_s3_path
params['baseline_data'] = baseline_data_output_s3_path
params['experiment_name'] = experiment.name

In [ ]:
s3_client = boto3.client("s3", region_name=region)
step_name = "02-preprocess"
write_params(s3_client, step_name, params, notebook_param_s3_bucket_prefix)

In [51]:
mlflow.end_run()